In [3]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
import pyspark.sql.functions as F
from pyspark.sql.types import *
from notebookutils import mssparkutils
import pandas as pd

# Verificar versión de Spark
print(f"Versión de Spark: {spark.version}")

StatementMeta(, 831dbd63-0b2b-4e58-9714-d4f50c27d911, 5, Finished, Available, Finished)

Versión de Spark: 3.5.1.5.4.20250914.1


In [4]:
# Leer los archivos CSV con los esquemas definidos
# Este comando lee los archivos CSV utilizando los esquemas definidos anteriormente

try:
    # Lectura desde capa Silver
    df_clientes_silver = spark.read.format("delta").load("Files/Silver/clientes_silver")
    df_brokers_silver = spark.read.format("delta").load("Files/Silver/brokers_silver")
    df_projects_silver = spark.read.format("delta").load("Files/Silver/projects_silver")
    df_properties_silver = spark.read.format("delta").load("Files/Silver/properties_silver")
    df_campaigns_silver = spark.read.format("delta").load("Files/Silver/campaigns_silver")
    df_leads_silver = spark.read.format("delta").load("Files/Silver/leads_silver")
    df_sales_silver = spark.read.format("delta").load("Files/Silver/sales_silver")

    print("✅ Archivos parquet leídos correctamente")
except Exception as e:
    print(f"Error al leer archivos: {str(e)}")

StatementMeta(, 831dbd63-0b2b-4e58-9714-d4f50c27d911, 6, Finished, Available, Finished)

✅ Archivos parquet leídos correctamente


In [5]:
# Realizar transformaciones en los dataframes
# Aplicamos transformaciones básicas para mejorar la calidad de los datos

# Transformación de clientes: Añadir columna nombre_completo
df_clientes_procesado = df_clientes_silver.withColumn(
    "nombre_completo", 
    F.concat(F.col("FirstName"), F.lit(" "), F.col("LastName"))
)

# Transformación de brokers: Añadir columna correo_completo
df_brokers_procesado = df_brokers_silver.withColumn(
    "correo_completo", 
    F.concat(F.col("Email"), F.lit("@email.com"))
)

# Transformación de projects: Añadir columna city-region
df_projects_procesado = df_projects_silver.withColumn(
    "city-region", 
    F.concat(F.col("City"), F.lit("-"),F.col("Region"))
)

# Transformación de properties: Añadir columna precioxmetro
df_properties_procesado = df_properties_silver.withColumn(
    "precioxmetro", 
    F.round((F.col("ListPriceUSD")/F.col("Size_m2")),2)
)

# Transformación de campaigns: Añadir columna diferencia_dias
df_campaigns_procesado = df_campaigns_silver.withColumn(
    "diferencia_dias", 
    F.datediff(F.col("EndDate"),F.col("StartDate"))
)

# Transformación de leads: Añadir columna año
df_leads_procesado = df_leads_silver.withColumn(
    "Año", 
    F.year(F.col("LeadDate"))
)

# Transformación de sales: Añadir columna tipo de dia
df_sales_procesado = df_sales_silver.withColumn(
    "tipo_dia",
    F.when(F.dayofweek(F.col("SaleDate")) == 1, "Fin de semana")
    .when(F.dayofweek(F.col("SaleDate")) == 2,"Fin de semana")
        .otherwise("Dia de semana")
)

StatementMeta(, 831dbd63-0b2b-4e58-9714-d4f50c27d911, 7, Finished, Available, Finished)

In [6]:
# Exploración de los datos
# Mostramos una vista previa de los datos procesados

print("Vista previa de los datos de clientes:")
display(df_clientes_procesado.limit(5))

print("Vista previa de los datos de brokers:")
display(df_brokers_procesado.limit(5))

print("Vista previa de los datos de projects:")
display(df_projects_procesado.limit(5))

print("Vista previa de los datos de properties:")
display(df_properties_procesado.limit(5))

print("Vista previa de los datos de campaigns:")
display(df_campaigns_procesado.limit(5))

print("Vista previa de los datos de leads:")
display(df_leads_procesado.limit(5))

print("Vista previa de los datos de sales:")
display(df_sales_procesado.limit(5))

StatementMeta(, 831dbd63-0b2b-4e58-9714-d4f50c27d911, 8, Finished, Available, Finished)

Vista previa de los datos de clientes:


SynapseWidget(Synapse.DataFrame, 962afee9-0042-42e1-a75d-553cacd69ce9)

Vista previa de los datos de brokers:


SynapseWidget(Synapse.DataFrame, 813a341d-0c3e-4fcb-896d-608596649169)

...

SynapseWidget(Synapse.DataFrame, 3144ed17-0bf3-476d-a895-c8ae3894d6cc)

Vista previa de los datos de leads:


SynapseWidget(Synapse.DataFrame, 081452a2-d986-4332-a0b5-0496aa6fd057)

Vista previa de los datos de sales:


SynapseWidget(Synapse.DataFrame, ca107ccf-3e7e-4bb6-b801-ec3801e74fe7)

In [7]:
# Guardar los dataframes como tablas Delta
# Guardamos los dataframes procesados como tablas Delta en el lakehouse

try:
    # Guardar clientes
    df_clientes_procesado.write.format("delta").mode("overwrite").save("Files/Golden/clientes_gold")
    
    # Guardar brokers
    df_brokers_procesado.write.format("delta").mode("overwrite").save("Files/Golden/brokers_gold")
    
    # Guardar projects
    df_projects_procesado.write.format("delta").mode("overwrite").save("Files/Golden/projects_gold")
    
    # Guardar properties
    df_properties_procesado.write.format("delta").mode("overwrite").save("Files/Golden/properties_gold")
    
    # Guardar campaigns
    df_campaigns_procesado.write.format("delta").mode("overwrite").save("Files/Golden/campaigns_gold")

    # Guardar leads
    df_leads_procesado.write.format("delta").mode("overwrite").save("Files/Golden/leads_gold")
    
    # Guardar sales
    df_sales_procesado.write.format("delta").mode("overwrite").save("Files/Golden/sales_gold")
    
    print("✅ Dataframes guardados como archivos Delta correctamente en gold")
except Exception as e:
    print(f"Error al guardar archivos Delta: {str(e)}")

StatementMeta(, 831dbd63-0b2b-4e58-9714-d4f50c27d911, 9, Finished, Available, Finished)

✅ Dataframes guardados como archivos Delta correctamente
